## 필요한 라이브러리 가져오기

In [1]:
# soynlp
# pandas
# gensim

import pandas as pd
from tqdm import tqdm

## 데이터 가져오기

In [2]:
# https://github.com/e9t/nsmc/
# nsmc 데이터 다운로드

dataset = pd.read_csv("ratings.txt", sep="\t")

In [3]:
dataset.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [4]:
print(len(dataset))

200000


## 데이터셋 Null 확인

In [5]:
print(dataset.isnull().sum())

id          0
document    8
label       0
dtype: int64


In [6]:
dataset.loc[dataset.document.isnull()]

,id,document,label
46471,6369843,NaN,1
60735,511097,NaN,1
77665,2172111,NaN,1
84098,402110,NaN,1
127017,5942978,NaN,0
172375,5026896,NaN,0
173526,1034280,NaN,0
197279,1034283,NaN,0


In [7]:
# null 값 제거

dataset = dataset.dropna(how='any')
print(dataset.isnull().values.any())

False


## 중복 제거

In [8]:
# document 컬럼의 중복된 값이 있는지 확인
dataset['document'].nunique()

194543

In [9]:
# 중복 제거
print("중복 제거 전 데이터 갯수 : ",len(dataset))
dataset.drop_duplicates(subset=['document'], inplace=True)
print("중복 제거 후 데이터 갯수 : ",len(dataset))

중복 제거 전 데이터 갯수 :  199992
중복 제거 후 데이터 갯수 :  194543


## 전처리

In [10]:
dataset.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [11]:
# 정제하기
dataset['document'] = dataset['document'].str.replace("[^ㄱ-하ㅣ가-힣]"," ")

C:\Users\USER\AppData\Local\Temp\ipykernel_10952\1749395364.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['document'] = dataset['document'].str.replace("[^ㄱ-하ㅣ가-힣]"," ")


In [12]:
dataset.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [13]:
# 토큰화

In [14]:
# 한국어 토큰화 라이브러리는 nltk, konlpy, soynlp 등 이 있음.
# konlpy를 많이 사용하지만 Windows환경에서는 konlpy 사용 불가. 
# soynlp 사용. 
# 설명은 https://github.com/lovit/soynlp 참고

from soynlp.noun import LRNounExtractor
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer


In [15]:
noun_extractor = LRNounExtractor()
nouns = noun_extractor.train_extract(list(dataset['document'])) # list of str like

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (66303, 37289) tokens
[Noun Extractor] building L-R graph was done000 / 194543 sents
[Noun Extractor] 13747 nouns are extracted


In [16]:
word_extractor = WordExtractor(
    min_frequency=50, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

In [17]:
word_extractor.train(list(dataset['document']))
words = word_extractor.extract()

training was done. used memory 0.895 Gbse memory 0.757 Gb
all cohesion probabilities was computed. # words = 11119
all branching entropies was computed # words = 103128
all accessor variety was computed # words = 103128


In [18]:
cohesion_score = {word:score.cohesion_forward for word, score in words.items()}

noun_scores = {noun:score.score for noun, score in nouns.items()}
combined_scores = {noun:score + cohesion_score.get(noun, 0)
    for noun, score in noun_scores.items()}
combined_scores.update(
    {subword:cohesion for subword, cohesion in cohesion_score.items()
    if not (subword in combined_scores)}
)

tokenizer = LTokenizer(scores=combined_scores)

In [19]:
temp_text = dataset['document'][1]
print(str(temp_text))
print(tokenizer.tokenize(str(temp_text)))

디자인을 배우는 학생으로  외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데  사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다 
['디자인', '을', '배우', '는', '학생', '으로', '외국', '디자이너와', '그들', '이', '일군', '전통', '을', '통해', '발전', '해가는', '문화', '산업이', '부러웠는데', '사실', '우리나라', '에서도', '그', '어려운', '시절에', '끝까지', '열정', '을', '지킨', '노라', '노', '같은', '전통', '이있어', '저와', '같은', '사람들', '이', '꿈을', '꾸고', '이뤄나갈', '수', '있다는', '것에', '감사', '합니다']


In [20]:
result = []
for line in tqdm(dataset['document'].tolist()):
    result.append(tokenizer.tokenize(str(line)))


100%|███████████████████████████████████████████████████████████████████████| 194543/194543 [00:05<00:00, 35137.19it/s]


In [21]:
result[0]

['어릴때', '보고', '지금', '다시봐도', '재밌', '어요ㅋㅋ']

## 모델 학습

In [22]:
# gensim에서는 word2vec 라이브러리 제공
from gensim.models import Word2Vec
model = Word2Vec(result, window=5, min_count=5, workers=4, sg=0)

## 결과 확인

In [23]:
model_result1 = model.wv.most_similar("대한민국")
print(model_result1)

[('조선', 0.8169139623641968), ('국가', 0.813879668712616), ('국민', 0.8132150173187256), ('미국', 0.8029162287712097), ('아시아', 0.7863821387290955), ('선동', 0.7859699726104736), ('인류', 0.7729686498641968), ('북한', 0.7716014385223389), ('독일', 0.7648884057998657), ('국내', 0.7629224061965942)]


In [24]:
model_result2 = model.wv.most_similar("어벤져스")
print(model_result2)

[('매드맥스', 0.8953632116317749), ('신세계', 0.8853313326835632), ('차에서', 0.8801828622817993), ('월드', 0.8760780096054077), ('토이스토리', 0.8687630891799927), ('트랜스포머', 0.8666226267814636), ('착신아리', 0.8642228841781616), ('여중생', 0.8630887269973755), ('소라', 0.8619428277015686), ('원피스', 0.8619065284729004)]


In [25]:
model_result3 = model.wv.most_similar("학교")
print(model_result3)

[('초등', 0.8931469321250916), ('학년', 0.885145902633667), ('학교때', 0.8519147038459778), ('고등학교', 0.8465683460235596), ('학년때', 0.8367843627929688), ('보던', 0.8161652684211731), ('중학교', 0.7957607507705688), ('초', 0.7797496914863586), ('살때', 0.7729234099388123), ('빌려', 0.7670618891716003)]
